In [ ]:
#| hide
#| default_exp utils

In [ ]:
#| export
# dill is an improved version of pickle, using it to support namedtuples
import dill
from pathlib import Path

In [ ]:
#| export
# inspired from https://gist.github.com/shantanuo/c6a376309d6bac6bd55bf77e3961b5fb
def cache_disk(base_file):
    base_file = Path(base_file)
    def decorator(original_func):
        
        # take the hash of the function content
        filename = base_file.parent / (base_file.stem + str(hash(original_func.__code__.co_code)) + ".pickle")
        
        try:
            cache = dill.load(open(filename, 'rb'))
        except (IOError, ValueError):
            cache = {}

        def save_data():
            dill.dump(cache, open(filename, "wb"))  

        def new_func(*args):
            if tuple(args) not in cache:
                cache[tuple(args)] = original_func(*args)
                save_data()
            return cache[args]

        return new_func

    return decorator

In [ ]:
import time
from tempfile import tempdir

In [ ]:
@cache_disk(Path(tempdir) / "test_cache")
def slow_add(a,b):
    time.sleep(1)
    return a + b 

this time is the first time so not from the cache

In [ ]:
%time slow_add(1,2)

CPU times: user 0 ns, sys: 1.43 ms, total: 1.43 ms
Wall time: 1 s


3

now is much faster beacuse of the cache

In [ ]:
%time slow_add(1,2)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.48 µs


3

adding comments doesn't change the hash, so the function is still cached

In [ ]:
@cache_disk("test_cache")
def slow_add(a,b):
    time.sleep(1)
    # this is a comment
    return a + b 

In [ ]:
%time slow_add(1,2)

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 5.25 µs


3

while changing the code changes the hash, so there is no cache

In [ ]:
@cache_disk("test_cache")
def slow_add(a,b):
    time.sleep(1)
    return b + a 

In [ ]:
%time slow_add(1,2)

CPU times: user 1.13 ms, sys: 56 µs, total: 1.19 ms
Wall time: 1 s


3

In [ ]:
#| hide
from nbdev import nbdev_export
nbdev_export()